# QR Decomposition: Theory and Implementation

## Introduction

QR decomposition (also called QR factorization) is a fundamental matrix factorization technique in numerical linear algebra. It decomposes a matrix $A$ into the product of two matrices:

$$A = QR$$

where:
- $Q$ is an **orthogonal matrix** (i.e., $Q^T Q = I$)
- $R$ is an **upper triangular matrix**

## Mathematical Foundation

### Orthogonal Matrices

A matrix $Q \in \mathbb{R}^{m \times n}$ is orthogonal if its columns form an orthonormal set:

$$\mathbf{q}_i^T \mathbf{q}_j = \delta_{ij} = \begin{cases} 1 & \text{if } i = j \\ 0 & \text{if } i \neq j \end{cases}$$

This implies $Q^T Q = I$, and for square matrices, $Q^{-1} = Q^T$.

### The Gram-Schmidt Process

The classical Gram-Schmidt process constructs an orthonormal basis from linearly independent vectors. Given vectors $\{\mathbf{a}_1, \mathbf{a}_2, \ldots, \mathbf{a}_n\}$, we compute:

$$\mathbf{u}_k = \mathbf{a}_k - \sum_{j=1}^{k-1} \text{proj}_{\mathbf{q}_j}(\mathbf{a}_k)$$

$$\mathbf{q}_k = \frac{\mathbf{u}_k}{\|\mathbf{u}_k\|}$$

where the projection is:

$$\text{proj}_{\mathbf{q}_j}(\mathbf{a}_k) = \frac{\mathbf{q}_j^T \mathbf{a}_k}{\mathbf{q}_j^T \mathbf{q}_j} \mathbf{q}_j = (\mathbf{q}_j^T \mathbf{a}_k) \mathbf{q}_j$$

### Construction of R

The upper triangular matrix $R$ contains the projection coefficients:

$$r_{ij} = \begin{cases} \mathbf{q}_i^T \mathbf{a}_j & \text{if } i \leq j \\ 0 & \text{if } i > j \end{cases}$$

## Applications

QR decomposition is essential for:
- **Solving least squares problems**: $\min_x \|Ax - b\|_2$
- **Computing eigenvalues**: QR algorithm iteratively applies QR decomposition
- **Solving linear systems**: More numerically stable than LU for ill-conditioned matrices

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

np.random.seed(42)
np.set_printoptions(precision=4, suppress=True)

## Implementation: Classical Gram-Schmidt

We implement the classical Gram-Schmidt algorithm to compute the QR decomposition.

In [ ]:
def classical_gram_schmidt(A):
    """
    Compute QR decomposition using Classical Gram-Schmidt.
    
    Parameters:
    -----------
    A : ndarray of shape (m, n)
        Input matrix with m >= n
        
    Returns:
    --------
    Q : ndarray of shape (m, n)
        Orthogonal matrix
    R : ndarray of shape (n, n)
        Upper triangular matrix
    """
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    
    for j in range(n):
        # Start with the j-th column of A
        v = A[:, j].copy()
        
        # Subtract projections onto previous q vectors
        for i in range(j):
            R[i, j] = np.dot(Q[:, i], A[:, j])
            v = v - R[i, j] * Q[:, i]
        
        # Normalize
        R[j, j] = np.linalg.norm(v)
        if R[j, j] > 1e-10:
            Q[:, j] = v / R[j, j]
        else:
            Q[:, j] = v
    
    return Q, R

# Example: 4x3 matrix
A = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 7],
    [10, 11, 12]
], dtype=float)

print("Original Matrix A:")
print(A)
print(f"\nShape: {A.shape}")

In [ ]:
# Compute QR decomposition
Q, R = classical_gram_schmidt(A)

print("Orthogonal Matrix Q:")
print(Q)
print(f"\nUpper Triangular Matrix R:")
print(R)

## Verification of Orthogonality

We verify that $Q^T Q = I$ and $A = QR$.

In [ ]:
# Check orthogonality: Q^T Q should be identity
QtQ = Q.T @ Q
print("Q^T Q (should be identity):")
print(QtQ)

print(f"\nOrthogonality error: {np.linalg.norm(QtQ - np.eye(3)):.2e}")

# Check reconstruction: A = QR
A_reconstructed = Q @ R
print(f"\nReconstruction error ||A - QR||: {np.linalg.norm(A - A_reconstructed):.2e}")

## Comparison with NumPy's Implementation

NumPy uses Householder reflections, which is more numerically stable than Gram-Schmidt.

In [ ]:
# NumPy's QR (uses Householder reflections)
Q_np, R_np = np.linalg.qr(A)

print("NumPy Q:")
print(Q_np)
print("\nNumPy R:")
print(R_np)

# Note: Signs may differ (both are valid QR decompositions)
print(f"\nNumPy reconstruction error: {np.linalg.norm(A - Q_np @ R_np):.2e}")

## Application: Solving Least Squares

The least squares problem $\min_x \|Ax - b\|_2$ can be solved efficiently using QR decomposition:

$$Ax = b \implies QRx = b \implies Rx = Q^T b$$

Since $R$ is upper triangular, we can solve by back substitution.

In [ ]:
def solve_least_squares_qr(A, b):
    """
    Solve least squares problem using QR decomposition.
    """
    Q, R = np.linalg.qr(A)
    # Solve Rx = Q^T b by back substitution
    y = Q.T @ b
    x = np.linalg.solve(R, y)
    return x

# Generate overdetermined system
m, n = 100, 3
A_ls = np.column_stack([np.ones(m), np.linspace(0, 10, m), np.linspace(0, 10, m)**2])
x_true = np.array([1, 2, -0.5])
b_ls = A_ls @ x_true + 0.5 * np.random.randn(m)

# Solve using QR
x_qr = solve_least_squares_qr(A_ls, b_ls)
print(f"True coefficients: {x_true}")
print(f"Estimated coefficients: {x_qr}")
print(f"Error: {np.linalg.norm(x_true - x_qr):.4f}")

## Visualization

We visualize the QR decomposition in 3D, showing how the original column vectors are transformed into orthonormal vectors.

In [ ]:
# Create a simple 3x3 matrix for visualization
A_3d = np.array([
    [2, 1, 1],
    [1, 3, 1],
    [1, 1, 4]
], dtype=float)

Q_3d, R_3d = np.linalg.qr(A_3d)

# Create figure with subplots
fig = plt.figure(figsize=(14, 5))

# Plot 1: Matrix structure visualization
ax1 = fig.add_subplot(131)
matrices = [A_3d, Q_3d, R_3d]
titles = ['A (Original)', 'Q (Orthogonal)', 'R (Upper Triangular)']

# Combined heatmap
combined = np.zeros((3, 9))
combined[:, 0:3] = A_3d
combined[:, 3:6] = Q_3d
combined[:, 6:9] = R_3d

im = ax1.imshow(combined, cmap='RdBu', aspect='equal', vmin=-4, vmax=4)
ax1.set_xticks([1, 4, 7])
ax1.set_xticklabels(['A', 'Q', 'R'])
ax1.set_yticks([])
ax1.axvline(x=2.5, color='black', linewidth=2)
ax1.axvline(x=5.5, color='black', linewidth=2)
plt.colorbar(im, ax=ax1, fraction=0.046)
ax1.set_title('QR Decomposition: A = QR')

# Plot 2: Orthogonality verification (Q^T Q)
ax2 = fig.add_subplot(132)
QtQ_3d = Q_3d.T @ Q_3d
im2 = ax2.imshow(QtQ_3d, cmap='RdBu', vmin=-0.5, vmax=1.5)
for i in range(3):
    for j in range(3):
        ax2.text(j, i, f'{QtQ_3d[i, j]:.2f}', ha='center', va='center', fontsize=10)
ax2.set_title(r'$Q^T Q$ (Identity Matrix)')
ax2.set_xticks(range(3))
ax2.set_yticks(range(3))
plt.colorbar(im2, ax=ax2, fraction=0.046)

# Plot 3: Least squares fit
ax3 = fig.add_subplot(133)
t = np.linspace(0, 10, m)
ax3.scatter(t, b_ls, alpha=0.5, s=20, label='Noisy data')
y_fit = A_ls @ x_qr
ax3.plot(t, y_fit, 'r-', linewidth=2, label='QR least squares fit')
y_true = A_ls @ x_true
ax3.plot(t, y_true, 'g--', linewidth=1.5, label='True curve')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_title('Least Squares via QR Decomposition')
ax3.legend(fontsize=8)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to plot.png")

## Numerical Stability Analysis

The classical Gram-Schmidt algorithm suffers from numerical instability. Let's compare it with the Modified Gram-Schmidt algorithm on an ill-conditioned matrix.

In [ ]:
def modified_gram_schmidt(A):
    """
    Compute QR decomposition using Modified Gram-Schmidt.
    More numerically stable than classical Gram-Schmidt.
    """
    m, n = A.shape
    Q = A.copy().astype(float)
    R = np.zeros((n, n))
    
    for i in range(n):
        R[i, i] = np.linalg.norm(Q[:, i])
        if R[i, i] > 1e-10:
            Q[:, i] = Q[:, i] / R[i, i]
        
        for j in range(i + 1, n):
            R[i, j] = np.dot(Q[:, i], Q[:, j])
            Q[:, j] = Q[:, j] - R[i, j] * Q[:, i]
    
    return Q, R

# Create ill-conditioned matrix (Hilbert matrix)
def hilbert_matrix(n):
    return np.array([[1/(i + j + 1) for j in range(n)] for i in range(n)])

n = 8
H = hilbert_matrix(n)
print(f"Condition number of {n}x{n} Hilbert matrix: {np.linalg.cond(H):.2e}")

# Compare orthogonality errors
Q_cgs, R_cgs = classical_gram_schmidt(H)
Q_mgs, R_mgs = modified_gram_schmidt(H)
Q_np, R_np = np.linalg.qr(H)

err_cgs = np.linalg.norm(Q_cgs.T @ Q_cgs - np.eye(n))
err_mgs = np.linalg.norm(Q_mgs.T @ Q_mgs - np.eye(n))
err_np = np.linalg.norm(Q_np.T @ Q_np - np.eye(n))

print(f"\nOrthogonality error ||Q^T Q - I||:")
print(f"  Classical Gram-Schmidt: {err_cgs:.2e}")
print(f"  Modified Gram-Schmidt:  {err_mgs:.2e}")
print(f"  NumPy (Householder):    {err_np:.2e}")

## Conclusion

QR decomposition is a powerful tool in numerical linear algebra with key properties:

1. **Uniqueness**: For a matrix $A$ with full column rank, the QR decomposition with positive diagonal entries in $R$ is unique

2. **Numerical Stability**: 
   - Modified Gram-Schmidt is more stable than classical Gram-Schmidt
   - Householder reflections (used by NumPy) are the most stable

3. **Computational Cost**: $\mathcal{O}(mn^2)$ for an $m \times n$ matrix

4. **Applications**: Solving least squares, computing eigenvalues (QR algorithm), and orthogonalization of vectors

The choice of algorithm depends on the trade-off between computational efficiency and numerical stability required for the specific application.